### Import Libraries

In [1]:
from encoder.encoder import Encoder
from retriever.vector_db import VectorDatabase
from langchain.docstore.document import Document
from generator.generator import Generator
from translator.translator import Translator
from classifier.language_classifier import LanguageDetector
import pandas as pd


generator = Generator()
encoder = Encoder()
vectordb = VectorDatabase(encoder.encoder)
translator = Translator()
lang_classifier = LanguageDetector()

/Users/minp/miniconda3/envs/large-language-models/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/minp/miniconda3/envs/large-language-models/lib/python3.10/site-packages/langchain/utils/utils.py:159: UserWarning: WARNING! encoding is not default parameter.
                encoding was transferred to model_kwargs.
                Please confirm that encoding is what you intended.
  warnings.warn(
llama_model_loader: loaded meta data with 19 key-value pairs and 291 tensors from /Users/minp/AI_LLM/large-language-models/translation/src/../../model/nous-hermes-llama-2-7b.Q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q4_0     [  4096,  4096,     1

### Load Data and insert into vector database

In [2]:
df = pd.read_csv("data/data.csv")
df["full_review"] = df[["reviews.title", "reviews.text"]].apply(
    lambda row: ". ".join(row.values.astype(str)), axis=1
)
for product_id in df["asins"].unique()[:10]:
    # create documents to store in Postgres
    docs = [
        Document(page_content=item)
        for item in df[df["asins"] == product_id]["full_review"].tolist()
    ]

    passages = vectordb.create_passages_from_documents(docs)
    vectordb.store_passages_db(passages, product_id)

FileNotFoundError: [Errno 2] No such file or directory: 'data/data.csv'

### Query Database

In [ ]:
QUERY = "B00QJDU3KY|Qu'est-ce que les gens aiment dans le produit?"
ID = QUERY.split("|")[0]
QUERY = QUERY.split("|")[1]

In [ ]:
# detect customer language to reply back in same language
user_detected_language = lang_classifier.detect_language(QUERY)

# retrieve context about the product
context = vectordb.retrieve_most_similar_document(QUERY, k=4, id=ID)

# convert all context into english for our LLM
english_context = []
for doc in context:
    detected_language = lang_classifier.detect_language(doc)
    if detected_language != "en_XX":
        doc = translator.translate(doc, detected_language, "en_XX")
    english_context.append(doc)

context = "\n".join(english_context)

### Generate answer

In [ ]:
# translate customer query to english
if user_detected_language != "en_XX":
    QUERY = translator.translate(QUERY, user_detected_language, "en_XX")

# generate anwser with our LLM based on customer query and context
answer = generator.get_answer(context, QUERY)
print(answer)

# if the customer language is not english, then translate it
if user_detected_language != "en_XX":
    answer = translator.translate(answer, "en_XX", user_detected_language)
print(answer)

Llama.generate: prefix-match hit


 People love the Kindle Paperwhite because it is lightweight and easy to hold for long periods of time. The screen is also clear and easy to read even in low light conditions, making it perfect for reading at night or on cloudy days. Additionally, the battery life is impressive, allowing users to go for extended periods without needing a charge. Overall, people find the Kindle Paperwhite to be an excellent choice for anyone who loves to read and wants a device that is easy to use and convenient to carry around.



llama_print_timings:        load time =   13934.09 ms
llama_print_timings:      sample time =      32.71 ms /   112 runs   (    0.29 ms per token,  3423.82 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   40370.03 ms /   112 runs   (  360.45 ms per token,     2.77 tokens per second)
llama_print_timings:       total time =   40754.62 ms
/Users/rafael/miniconda3/envs/zaai/lib/python3.10/site-packages/transformers/generation/utils.py:1353: UserWarning: Using `max_length`'s default (200) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
